In [1]:
import numpy as np
#from tqdm import tqdm 
from matplotlib import pyplot as plt

In [2]:
def initialisation(dims):
    n=len(dims)
    parametres={}
    for i in range(1,n):
        parametres["W"+str(i)]=np.random.randn(dims[i],dims[i-1])
        parametres["b"+str(i)]=np.random.randn(dims[i],1)
    return parametres

In [3]:
parametres=initialisation([2,32,32,1])
for key,val in parametres.items():
    print(key, val.shape)

W1 (32, 2)
b1 (32, 1)
W2 (32, 32)
b2 (32, 1)
W3 (1, 32)
b3 (1, 1)


In [4]:
def forward_propagation(x,param):
    activations={'A0':x}
    n=len(param)//2
    for i in range(1,n+1):
        z=param["W"+str(i)].dot(activations["A"+str(i-1)])+param['b'+str(i)]
        activations["A"+str(i)]=1/(1+np.exp(-z))
    return activations

In [5]:
x=np.random.randn(2,100)

In [6]:
x.shape[0]

2

In [7]:
activations=forward_propagation(x,parametres)
for key,val in activations.items():
    print(key, val.shape)

A0 (2, 100)
A1 (32, 100)
A2 (32, 100)
A3 (1, 100)


In [8]:
def back_propagation(y,activ,param):
    n=len(param)//2
    m=y.shape[1]
    dZ=activ["A"+str(n)]-y
    gradients={}
    for i in range(n,0,-1):
        gradients['dW'+str(i)]=1/m*np.dot(dZ,activ['A'+str(i-1)].T)
        gradients['db'+str(i)]=1/m*np.sum(dZ,axis=1,keepdims=True)
        if i>=1:
            dZ=np.dot(param['W'+str(i)].T,dZ)*activ['A'+str(i-1)]*(1-activ['A'+str(i-1)])
        
    return gradients

In [9]:
y=np.random.randint(0,2,(1,100))
y

array([[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1]])

In [10]:
gradients=back_propagation(y,activations,parametres)
for key,val in gradients.items():
    print(key, val.shape)

dW3 (1, 32)
db3 (1, 1)
dW2 (32, 32)
db2 (32, 1)
dW1 (32, 2)
db1 (32, 1)


In [11]:
def update(grad,param,learning_rate):
    n=len(param)//2
    for i in range(1,n+1):
        param['W'+str(i)]=param['W'+str(i)]-learning_rate*grad['dW'+str(i)]
        param['b'+str(i)]=param['b'+str(i)]-learning_rate*grad['db'+str(i)]
    return param

In [12]:
def loss_function(y,a):
    m=y.shape[1]
    arr=1.e-15
    return np.sum(-y*np.log(a+arr)-(1-y)*np.log(1-a+arr))/m

In [13]:
def predict(x,parametres):
    activations=forward_propagation(x,parametres)
    n=len(parametres)//2
    return activations['A'+str(n)]

In [14]:
def artificial_neural_network(x_train,y_train,x_test,y_test,hidden_layers,learning_rate=0.01,n_iter=1000):
    np.random.seed(0)
    #initialisation des parametres du modele
    dimensions=list(hidden_layers)
    dimensions=[x_train.shape[0]]+dimensions
    dimensions+=[y_train.shape[0]]
    parametres=initialisation(dimensions)
    #Apprentissage du modèle
    train_loss=[]
    test_win=[]
    for i in tqdm(range(n_iter)):
        activations=forward_propagation(x_train,parametres)
        last_layer=len(parametres)//2
        gradients=back_propagation(y_train,activations,parametres)
        parametres=update(gradients,parametres,learning_rate)
        train_loss+=[loss_function(y_train,activations['A'+str(last_layer)])]
        y_pred=predict(x_test,parametres)
        y_pred=y_pred>0.5
        test_win+=[100*np.sum(y_pred==y_test)/y_test.shape[1]]
    #Visualisation de l'apprentissage
    plt.figure()
    plt.subplot(1,2,1)
    plt.plot(train_loss,c='r')
    plt.title('train loss')
    plt.subplot(1,2,2)
    plt.plot(test_win,c='g')
    plt.title('test win prec')
    plt.savefig("train_loss.png")
    plt.close()
    return parametres

In [15]:
import h5py
import numpy as np


def load_data():
    train_dataset = h5py.File('datasets/trainset.hdf5', "r")
    X_train = np.array(train_dataset["X_train"][:]) 
    y_train = np.array(train_dataset["Y_train"][:]) 

    test_dataset = h5py.File('datasets/testset.hdf5', "r")
    X_test = np.array(test_dataset["X_test"][:])
    y_test = np.array(test_dataset["Y_test"][:]) 
    
    return X_train, y_train, X_test, y_test

ModuleNotFoundError: No module named 'h5py'

In [17]:
X_train, y_train, X_test, y_test=load_data()

In [18]:
X_test.shape

(200, 64, 64)

In [19]:
y_train=y_train.reshape(1,y_train.shape[0])
y_test=y_test.reshape(1,y_test.shape[0])

In [20]:
#Flatten image
X_train=X_train.reshape(-1,X_train.shape[0])/X_train.max()
X_test=X_test.reshape(-1,X_test.shape[0])/X_train.max()

In [21]:
X_test.shape

(4096, 200)

In [37]:
#parametres=artificial_neural_network(X_train,y_train,X_test,y_test,(5000,5000,5000,5000))

In [38]:
#y_pred=predict(X_test,parametres)

In [24]:
y_pred=y_pred>0.5

In [25]:
100*np.sum(y_pred==y_test)/y_test.shape[1]

47.5

In [ ]:
plt.subplot(1,2,1)
plt.plot(y)
plt.subplot(1,2,2)
plt.plot(y)
plt.show()